In [53]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from datetime import date
import time
from lightgbm import LGBMClassifier
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
import joblib
from collections import Counter

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# DATA_PATH = "/content/drive/MyDrive/Jupyter/unlimited_power/pickles"
# infile = open(DATA_PATH+'/LGBM_CLF.pkl','rb')
model = joblib.load('/content/drive/MyDrive/Jupyter/unlimited_power/pickles/LGBM_CLF.pkl')

In [56]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 5                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [57]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_left]
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)

  dataset2 = dataset.drop('Time', axis = 1)
  return dataset2

def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def scale_to_supervised(train, n_in, dropnan=True):

    # input sequence (t-n, ... t-1)
    if dropnan:
      df = pd.DataFrame(train.iloc[:,:-1])
      df2 = pd.DataFrame(train.iloc[:,-1], index = train.index)
      cols, names = list(), list()
      future = [0]
      n_vars = 1 if type(df) is list else df.shape[1]
      n_vars2 = 1 if type(df2) is list else df2.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      for i in future:
          cols.append(df2.shift(-i))
          if i == 0:
              names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
          else:
              names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
    else:
      df = pd.DataFrame(train)
      cols, names = list(), list()
      n_vars = 1 if type(df) is list else df.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
      #agg.fillna(0,inplace=True)
    return agg

In [58]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [73]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    temp = pd.read_csv(file_path)
    testset2 = drop_clms(temp)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'sin_time', 'cos_time']]
    testset5 = scale_to_supervised(testset4, n_days, dropnan=False)
    pred = model.predict(testset5)
    test = Counter(list(pred))
    val_list = list()
    key_list = list()

    for key in test:
      key_list.append(key)
      val_list.append(test[key])
    temp['24'] = key_list[val_list.index(max(val_list))]
    
    
    temp.to_csv("/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/" + str(i) + '.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [59]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [61]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2['Goal'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'sin_time', 'cos_time','Goal']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
trainset = scale_to_supervised(dataset4, n_days)
trainset2 = trainset.iloc[:,:-1]
goals = trainset.iloc[:,-1]
X_train, X_valid, Y_train, Y_valid = train_test_split(trainset2,  goals, random_state=42, test_size=0.33)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [63]:
len(X_valid)

17266

In [64]:
model.predict(X_valid)

array([16, 13, 15, ..., 18,  6,  3])

In [68]:
Y_valid.values

array([16, 13, 15, ..., 18,  6,  3])

In [70]:
pred = model.predict(X_valid)

0.0


In [72]:
k = 0

for i in range(len(Y_valid.values)):
  predict = pred[i]
  real = Y_valid.values[i]
  real_b = real - 1
  real_t = real + 1
  if predict == real:
    k += 1

print(k/len(Y_valid.values))

0.9932815938839338
